In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU만 사용하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    except RuntimeError as e:
        # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
        print(e)

In [3]:
batch_size = 32
target_size=(200,200)

gen = ImageDataGenerator(rescale=1/255.)
test_gen = gen.flow_from_directory('./data/test/', target_size=target_size, class_mode='categorical', batch_size=batch_size, shuffle=False)
test_gen.reset()

Found 12240 images belonging to 12 classes.


In [4]:
model_list = [i for i in os.listdir('save_models') if i[0] != '.']
model_list

['InceptionResNetV2_12class',
 'Xception_12class',
 'EffNetV2M_12class',
 'VGG16_12class']

In [5]:
# model load
model = []
acc = []
pred = []

for mod in model_list:
    print('{}를 load 하고 있습니다.'.format(mod))
    model.append(tf.keras.models.load_model('./save_models/{}'.format(mod)))
    print('{} load 완료.'.format(mod))

InceptionResNetV2_12class를 load 하고 있습니다.
InceptionResNetV2_12class load 완료.
Xception_12class를 load 하고 있습니다.
Xception_12class load 완료.
EffNetV2M_12class를 load 하고 있습니다.
EffNetV2M_12class load 완료.
VGG16_12class를 load 하고 있습니다.
VGG16_12class load 완료.


In [6]:
for i in range(len(model)):
    print('{} model의 accuracy를 평가중입니다.'.format(model_list[i]))
    acc.append(model[i].evaluate(test_gen)[1]) # accuracy 저장 
    print('{} model의 predict를 진행중입니다.'.format(model_list[i]))
    pred.append(model[i].predict(test_gen)) # predict list 저장     
    
w_pred = np.zeros_like(pred[0]) #weighted pred 저장
for i in range(len(model)):
    w_pred += (pred[i]*acc[i])

InceptionResNetV2_12class model의 accuracy를 평가중입니다.
383/383 [==============================] - 31s 80ms/step - loss: 0.8112 - accuracy: 0.8718
InceptionResNetV2_12class model의 predict를 진행중입니다.
Xception_12class model의 accuracy를 평가중입니다.
383/383 [==============================] - 24s 62ms/step - loss: 0.3273 - accuracy: 0.9088
Xception_12class model의 predict를 진행중입니다.
EffNetV2M_12class model의 accuracy를 평가중입니다.
383/383 [==============================] - 47s 122ms/step - loss: 0.2086 - accuracy: 0.9482
EffNetV2M_12class model의 predict를 진행중입니다.
VGG16_12class model의 accuracy를 평가중입니다.
383/383 [==============================] - 22s 57ms/step - loss: 0.4851 - accuracy: 0.8631
VGG16_12class model의 predict를 진행중입니다.


In [7]:
#test gen의 image와 label 추출
n_img = test_gen.n
steps = n_img//batch_size
print(f'n_img : {n_img} \t steps : {steps}')

imgs, labels = [], []
for i in range(steps):
    a, b = test_gen.next()
    imgs.extend(a)
    labels.extend(b)

imgs = np.asarray(imgs)
labels = np.asarray(labels)

n_img : 12240 	 steps : 382


In [8]:
print(np.argmax(pred[0][1]),np.argmax(pred[1][1]),np.argmax(pred[2][1]),np.argmax(pred[3][1]),np.argmax(w_pred[1]))

0 0 0 0 0


In [9]:
pred[0][1].sum()

0.99999994

In [10]:
pred[1][1].sum()

0.9999999

In [11]:
pred[2][1].sum()

0.99999994

In [12]:
pred[3][1].sum()

0.99999994

In [15]:
w_pred[1].sum()

3.5919113

In [16]:
true=0
for i in range(len(imgs)):
    if np.argmax(w_pred[i])==np.argmax(labels[i]):
        true+=1
true

11598

In [17]:
11579/12240*100

94.59967320261438

In [24]:
print(*acc)
print(np.mean(acc))

0.8718137145042419 0.908823549747467 0.9482026100158691 0.8630719184875488
0.8979779481887817
